In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
sc = SparkContext(conf = conf)

def parseLine(line):
  fields = line.split(',')
  stationID=fields[0]
  entryType=fields[2]
  temperature=float(fields[3]) * 0.1 *(9.0 / 5.0) + 32.0
  return (stationID, entryType, temperature)
  
lines = sc.textFile("1800.csv")
parsedLines = lines.map(parseLine)
minTemps=parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps=minTemps.map(lambda x: (x[0], x[2]))
minTemps=stationTemps.reduceByKey(lambda x,y:min(x,y))
results = minTemps.collect()

for result in results:
  print(result[0] + "\t{:.2f}F".format(result[1]))

sc.stop()

ITE00100554	5.36F
EZE00100082	7.70F


In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("MaxTemperatures")
sc = SparkContext(conf = conf)

def parseLine(line):
  fields = line.split(',')
  stationID = fields[0]
  entryType = fields[2]
  temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
  return (stationID, entryType, temperature)
lines = sc.textFile("1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMAX" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: max(x,y))
results = minTemps.collect();
  
for result in results:
  print(result[0] + "\t{:.2f}F".format(result[1]))
  
sc.stop()

ITE00100554	90.14F
EZE00100082	90.14F


In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)
input = sc.textFile("book.txt")
words = input.flatMap(lambda x: x.split())
wordCounts = words.countByValue()

for word, count in wordCounts.items():
  cleanWord = word.encode('ascii', 'ignore')
  if (cleanWord):
    print(cleanWord.decode() + " " + str(count))

sc.stop()

In [ ]:
import re
from pyspark import SparkConf, SparkContext
def normalizeWords(text):
  return re.compile(r'\W+', re.UNICODE).split(text.lower())
  conf = SparkConf().setMaster("local").setAppName("WordCount")
  sc = SparkContext(conf = conf)
  input = sc.textFile("book.txt")
  words = input.flatMap(normalizeWords)
  wordCounts = words.countByValue()

for word, count in wordCounts.items():
  cleanWord = word.encode('ascii', 'ignore')
  if (cleanWord):
    print(cleanWord.decode() + " " + str(count))

sc.stop()

In [ ]:
import re
from pyspark import SparkConf, SparkContext
def normalizeWords(text):
  return re.compile(r'\W+', re.UNICODE).split(text.lower())
  
conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)

input = sc.textFile("book.txt")
words = input.flatMap(normalizeWords)
  
wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey(False)
results = wordCountsSorted.collect()

for result in results:
  count = str(result[0])
  word = result[1].encode('ascii', 'ignore')
  if (word):
    print(word.decode() + ":\t\t" + count)
  
sc.stop()

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("SpendByCustomerSorted")
sc = SparkContext(conf = conf)

def extractCustomerPricePairs(line):
  fields = line.split(',')
  return (int(fields[0]), float(fields[2]))
  
input = sc.textFile("customer-orders.csv")
mappedInput = input.map(extractCustomerPricePairs)
totalByCustomer = mappedInput.reduceByKey(lambda x, y: x + y)

results = totalByCustomer.collect();
for result in results:
  print(result)

sc.stop()

In [15]:
sc.stop()

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("PopularMovies")
sc = SparkContext(conf = conf)

lines = sc.textFile("u.data")
movies = lines.map(lambda x: (int(x.split()[1]), 1))
movieCounts = movies.reduceByKey(lambda x, y: x + y)

flipped = movieCounts.map( lambda xy : (xy[1], xy[0]) )
sortedMovies = flipped.sortByKey(False) #(False) Descending order
results = sortedMovies.collect()

for result in results:
  print(result)
  
sc.stop()

In [ ]:
sc.stop()


In [ ]:
from pyspark import SparkConf, SparkContext
def loadMovieNames():
  movieNames = {}
  with open("u.item", encoding = "ISO-8859-1") as f:
    for line in f:
      fields = line.split('|')
      movieNames[int(fields[0])] = fields[1]
    return movieNames
    
conf = SparkConf().setMaster("local").setAppName("PopularMovies")
sc = SparkContext(conf = conf)

nameDict = sc.broadcast(loadMovieNames())

lines = sc.textFile("u.data")
movies = lines.map(lambda x: (int(x.split()[1]), 1))
movieCounts = movies.reduceByKey(lambda x, y: x + y)

flipped = movieCounts.map(lambda xy : (xy[1], xy[0]))
sortedMovies = flipped.sortByKey(False)

sortedMoviesWithNames = sortedMovies.map(lambda countMovie : (nameDict.value[countMovie[1]], countMovie[0]))

results = sortedMoviesWithNames.collect()

for result in results:
  print(result)
sc.stop()

In [22]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("PopularHero")
sc = SparkContext(conf = conf)

def countCoOccurences(line):
  elements = line.split()
  return (int(elements[0]), len(elements) - 1)

def parseNames(line):
  fields = line.split('\"')
  return (int(fields[0]), fields[1].encode("utf8"))
  
names = sc.textFile("Marvel-names.txt")
namesRdd = names.map(parseNames)
lines = sc.textFile("Marvel-graph.txt")

pairings = lines.map(countCoOccurences)
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)
flipped = totalFriendsByCharacter.map(lambda xy : (xy[1], xy[0]))

mostPopular = flipped.max()
mostPopularName = namesRdd.lookup(mostPopular[1])[0]

print(str(mostPopularName) + " is the most popular superhero, with " + str(mostPopular[0]) + \
      " co-appearances.")

sc.stop()

b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.


In [20]:
sc.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import collections
# Create a SparkSession (Note, the config section is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "").appName("SparkSQL").getOrCreate()

def mapper(line):
  fields = line.split(',')
  return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), age=int(fields[2]), numFriends=int(fields[3]))

lines = spark.sparkContext.textFile("fakefriends.csv")
people = lines.map(mapper)
# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")
# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are RDDs and support all the normal RDD operations.
for teen in teenagers.collect():
  print(teen)
  # We can also use functions instead of SQL queries:
  
schemaPeople.groupBy("age").count().orderBy("age").show()

spark.stop()

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

def loadMovieNames():
  movieNames = {}
  with open("u.item", encoding = "ISO-8859-1") as f:
    for line in f:
      fields = line.split('|')
      movieNames[int(fields[0])] = fields[1]
    return movieNames
# Create a SparkSession (the config bit is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "").appName("PopularMovies").getOrCreate()
# Load up our movie ID -> name dictionary
nameDict = loadMovieNames()
# Get the raw data
lines = spark.sparkContext.textFile("u.data")
# Convert it to a RDD of Row objects
movies = lines.map(lambda x: Row(movieID =int(x.split()[1])))
# Convert that to a DataFrame
movieDataset = spark.createDataFrame(movies)
# Some SQL-style magic to sort all movies by popularity in one line!
topMovieIDs = movieDataset.groupBy("movieID").count().orderBy("count", ascending=False).cache()
# Show the results at this point:#|movieID|co

topMovieIDs.show()

top10=topMovieIDs.take(10)

print("\n")
for result in top10:
    print("%s: %d" % (nameDict[result[0]], result[1]))

spark.stop()

+-------+-----+
|movieID|count|
+-------+-----+
|     50|  584|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
|    286|  481|
|    288|  478|
|      1|  452|
|    300|  431|
|    121|  429|
|    174|  420|
|    127|  413|
|     56|  394|
|      7|  392|
|     98|  390|
|    237|  384|
|    117|  378|
|    172|  368|
|    222|  365|
|    313|  350|
+-------+-----+
only showing top 20 rows



Star Wars (1977): 584
Contact (1997): 509
Fargo (1996): 508
Return of the Jedi (1983): 507
Liar Liar (1997): 485
English Patient, The (1996): 481
Scream (1996): 478
Toy Story (1995): 452
Air Force One (1997): 431
Independence Day (ID4) (1996): 429
